In [2]:
using QuantumOptics

const N = 17 # Cavity Dimension

# Cavity Operators
fb = FockBasis(N-1)

a = destroy(fb)
at = create(fb)
Ic = identityoperator(fb)

# qutrit operators
qb = FockBasis(3-1)

Iq = identityoperator(qb)
q = destroy(qb)
qt = create(qb)
sz_ge = Operator(qb,qb, [-1 0 0; 0 1 0; 0 0 1])
sz_gf = Operator(qb,qb, [-1 0 0; 0 0 0; 0 0 1])
sy_ge = Operator(qb,qb, [0 -1im 0; 1im 0 0; 0 0 1])
sy_ef = Operator(qb,qb, [1 0 0; 0 0 -1im; 0 1im 0])
sx_ge = Operator(qb,qb, [0 1 0; 1 0 0; 0 0 1])
sx_ef = Operator(qb,qb, [1 0 0; 0 0 1; 0 1 0])




g = fockstate(qb,0)
e = fockstate(qb,1)
f = fockstate(qb,2)
ge = 1/√2*(g+e)
vac = fockstate(fb,0) 
proj_g = projector(g) ⊗ Ic ⊗ Ic ⊗ Ic
proj_e = projector(e) ⊗ Ic ⊗ Ic ⊗ Ic
proj_f = projector(f) ⊗ Ic ⊗ Ic ⊗ Ic

gvvv = g ⊗ vac ⊗ vac ⊗ vac
evvv = e ⊗ vac ⊗ vac ⊗ vac
gevvv = ((g+e)/√2) ⊗ vac ⊗ vac ⊗ vac
gfvvv = ((g+f)/√2) ⊗ vac ⊗ vac ⊗ vac


Ket(dim=14739)
  basis: [Fock(cutoff=2) ⊗ Fock(cutoff=16) ⊗ Fock(cutoff=16) ⊗ Fock(cutoff=16)]
 0.7071067811865475 + 0.0im
                0.0 + 0.0im
 0.7071067811865475 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                    ⋮
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im
                0.0 + 0.0im

In [3]:
function decode_pauli_string(oper_string, α)
    scale_factor_list = Float64[]
    displacement_list = ComplexF64[]
    is_imag_list = Bool[]

    for character in oper_string
        if character == 'X'
            push!(scale_factor_list,1)
            push!(displacement_list,2*α)
            push!(is_imag_list,false)
        elseif character == 'Y'
            push!(scale_factor_list,1)
            push!(displacement_list,-2*α)
            push!(is_imag_list,true)
        elseif character == 'Z'
            push!(scale_factor_list,1/2*(1/exp((1im*π/(4*α   ))^2/2)))
            push!(displacement_list,1im*π/(4*α))
            push!(is_imag_list,true)
        elseif character == 'I'
            push!(scale_factor_list,1/2)
            push!(displacement_list,0)
            push!(is_imag_list,false)
        else
            println("Invalid character")
        end
    end
    return scale_factor_list, displacement_list, is_imag_list
end

decode_pauli_string (generic function with 1 method)

In [9]:
function measure_3Mode_char_point(ψ0, is_imag_list ,displacement_list)
    if sum(is_imag_list)%2 == 1
        return expect(displace(fb,displacement_list[1]) ⊗ displace(fb,displacement_list[2]) ⊗ displace(fb,displacement_list[3]) ,ψ0).im
    else
        return expect(displace(fb,displacement_list[1]) ⊗ displace(fb,displacement_list[2]) ⊗ displace(fb,displacement_list[3]) ,ψ0).re
    end

    
end


function measure_3Mode_paulioper(ψ0, pauli_string, α)
    #println("entered measure_3Mode_paulioper")
    scale_factor_list, displacement_list, is_imag_list = decode_pauli_string(pauli_string, α)
    
    if sum(is_imag_list)%2 == 0
        #println("entered sum(is_imag_list)%2 == 0")
        displacement_list_2 = copy(displacement_list) 
        displacement_list_3 = copy(displacement_list)
        displacement_list_4 = copy(displacement_list)

        displacement_list_2[1] *= -1
        displacement_list_3[2] *= -1
        displacement_list_4[3] *= -1

        cf_point_1 = measure_3Mode_char_point(ψ0, is_imag_list, 
        displacement_list)

        cf_point_2 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_2)
        
        cf_point_3 = measure_3Mode_char_point(ψ0, is_imag_list, 
        displacement_list_3)

        cf_point_4 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_4)

        # taking care of the 4 cases 000, 110, 101, 011
        println("char func points:")
        println(cf_point_1)
        println(cf_point_2)
        println(cf_point_3)
        println(cf_point_4)
        

        return 2*prod(scale_factor_list)*1im^(sum(is_imag_list))*
        (cf_point_1 + (-1)^is_imag_list[1]*cf_point_2 + (-1)^is_imag_list[2]*cf_point_3 + (-1)^is_imag_list[3]*cf_point_4)

    else
            #println("entered sum(is_imag_list%2) == 1")
            displacement_list_2 = copy(displacement_list) 
            displacement_list_3 = copy(displacement_list)
            displacement_list_4 = copy(displacement_list)
    
            displacement_list_2[1] *= -1
            displacement_list_3[2] *= -1
            displacement_list_4[3] *= -1
    
            cf_point_1 = measure_3Mode_char_point(ψ0, is_imag_list, 
            displacement_list)
    
            cf_point_2 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_2)
            
            cf_point_3 = measure_3Mode_char_point(ψ0, is_imag_list, 
            displacement_list_3)
    
            cf_point_4 = measure_3Mode_char_point(ψ0, is_imag_list, displacement_list_4)
            println("char func points:")
            println(cf_point_1)
            println(cf_point_2)
            println(cf_point_3)
            println(cf_point_4)
        
           # case 111
            if sum(is_imag_list) == 3 
                return 2*prod(scale_factor_list)*(cf_point_1 + cf_point_2 + cf_point_3 +cf_point_4)
            else
                # case 001, 010, 100
                return 2*prod(scale_factor_list)*(-1)*
                (cf_point_1 + (-1)^is_imag_list[1]*cf_point_2 + (-1)^is_imag_list[2]*cf_point_3 + (-1)^is_imag_list[3]*cf_point_4)
            end
    
        end

end


function ideal_W_3ModeOper_expect(oper_string)
    
    tlb = SpinBasis(1//2)
    e_tl = spinup(tlb)
    g_tl = spindown(tlb)

    sx = sigmax(tlb)
    sy = sigmay(tlb)
    sz = sigmaz(tlb)
    Iq_tl = identityoperator(tlb)
    W_state = 1/√3*(e_tl ⊗ g_tl ⊗ g_tl + g_tl ⊗ e_tl ⊗ g_tl + g_tl ⊗ g_tl ⊗ e_tl)
    oper_dict = Dict('X' => sx, 'Y' => sy, 'Z' => sz, 'I' => Iq_tl)
    three_mode_oper = oper_dict[oper_string[1]] ⊗ oper_dict[oper_string[2]] ⊗ oper_dict[oper_string[3]]

    return real(expect(three_mode_oper, W_state))
end

ideal_W_3ModeOper_expect (generic function with 1 method)

In [5]:
function test_charfunc_pauli_measurements(pauli_string, α)
    println(pauli_string)
    W = 1/√3*(coherentstate(fb, -α) ⊗ coherentstate(fb, α) ⊗ coherentstate(fb, α) + coherentstate(fb, α) ⊗ coherentstate(fb, -α) ⊗ coherentstate(fb, α) + coherentstate(fb, α) ⊗ coherentstate(fb, α) ⊗ coherentstate(fb, -α))

    exp_result = measure_3Mode_paulioper(W, pauli_string, α)

    theoretical_result = ideal_W_3ModeOper_expect(pauli_string)
    print("experimental result: ")
    println(round(abs(exp_result), digits=3))
    print("theoretical result: ")
    println(round(abs(theoretical_result), digits=3))
    return exp_result, theoretical_result
end


test_charfunc_pauli_measurements (generic function with 1 method)

In [6]:
exp((1im*π/(4*1.8))^2/2)

0.9091974936303646 + 0.0im

In [11]:
α = 1.25

test_charfunc_pauli_measurements("IXX", α)

IXX


char func points:
0.00426778398841415
0.00426778398841415
0.3365535851019575
0.3365535851019577


experimental result: 0.682
theoretical result: 0.667


(0.6816427381807435 + 0.0im, 0.6666666666666669)

In [30]:

α = 1.8

# Loop over all possible Pauli string combinations
pauli_strings = ["I", "X", "Y", "Z"]
results = []
for pauli1 in pauli_strings
    for pauli2 in pauli_strings
        for pauli3 in pauli_strings
            pauli_string = string(pauli1, pauli2, pauli3)
            test_charfunc_pauli_measurements(pauli_string, α)
        end
    end
end


III


experimental result: 1.0
theoretical result: 1.0
IIX


experimental result: 0.005
theoretical result: 0.0
IIY


experimental result: 0.0
theoretical result: 0.0
IIZ


experimental result: 0.333
theoretical result: 0.333
IXI
experimental result: 

...

experimental result: 0.002
theoretical result: 0.0
ZXX


experimental result: 0.666
theoretical result: 0.667
ZXY


experimental result: 0.0
theoretical result: 0.0
ZXZ


experimental result: 0.001
theoretical result: 0.0
ZYI


OutOfMemoryError: OutOfMemoryError()